# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

#print(len(full_data_rows_list))
#print(full_data_rows_list)

# creating a smaller event data csv file called event_data_file_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### With Apache Cassandra you model the database tables on the queries you want to run. So, we are modeling tables for three queries below.

In [8]:
# Mapping column location from csv
# 0 --> artist_name
# 1 --> first_name
# 2 --> gender
# 3 --> item_in_session
# 4 --> last_name
# 5 --> length
# 6 --> level
# 7 --> location
# 8 --> session_id
# 9 --> song_title
# 10 --> user_id

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4



- Query 1 asks us to give artist, song title and song's length for particular session id and item in session. So we will use session id as partition key and item in session as clustering key to model our table and name it as song_details.

In [9]:
# Query to create the table
query = "CREATE TABLE IF NOT EXISTS song_details"
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length FLOAT, \
                PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
# Query to populated the data in the table from the csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO song_details(session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [11]:
# Query to get the results from the table

song_details_query = "SELECT artist_name, song_title, song_length FROM song_details WHERE session_id = %s AND item_in_session = %s"
try:
    rows = session.execute(song_details_query, (338, 4))
except Exception as e:
    print(e)
    
for row in rows:
    print("The artist_name is: {}".format(row.artist_name))
    print("The song_title is: {}".format(row.song_title))
    print("The song_length is: {}".format(row.song_length))
    print("******************************************************************************************************************************")

The artist_name is: Faithless
The song_title is: Music Matters (Mark Knight Dub)
The song_length is: 495.30731201171875
******************************************************************************************************************************


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

- Query 2 asks us to give artist/s, song title, user's first and last name for particular user id and session id. So we will use user id and session id as composite partition key and item in session as clustering key since we need data sorted by item in session to model our table and name it as user_details. 

In [12]:
# Query to create the table

query = "CREATE TABLE IF NOT EXISTS user_details"
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, first_name text, last_name text, \
                PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [13]:
# Query to populated the data in the table from the csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_details(user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [14]:
# Query to get the results from the table

user_details_query = "SELECT artist_name, song_title, first_name, last_name FROM user_details WHERE user_id = %s AND session_id = %s"
try:
    rows = session.execute(user_details_query, (10, 182))
except Exception as e:
    print(e)
    
for row in rows:
    print("The artist_name is: {}".format(row.artist_name))
    print("The song_title is: {}".format(row.song_title))
    print("The first_name is: {}".format(row.first_name))
    print("The last_name is: {}".format(row.last_name))
    print("******************************************************************************************************************************")

The artist_name is: Down To The Bone
The song_title is: Keep On Keepin' On
The first_name is: Sylvie
The last_name is: Cruz
******************************************************************************************************************************
The artist_name is: Three Drives
The song_title is: Greece 2000
The first_name is: Sylvie
The last_name is: Cruz
******************************************************************************************************************************
The artist_name is: Sebastien Tellier
The song_title is: Kilometer
The first_name is: Sylvie
The last_name is: Cruz
******************************************************************************************************************************
The artist_name is: Lonnie Gordon
The song_title is: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)
The first_name is: Sylvie
The last_name is: Cruz
***************************************************************************************************************

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

- Query 3 asks us to give user's first and last name is music app history who listened to the song All Hands Against His Own. Here we use song_title as partiton key and user_id as clustering key and create a table named event_details.

In [15]:
# Query to create the table

query = "CREATE TABLE IF NOT EXISTS event_details"
query = query + "(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
# Query to populated the data in the table from the csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO event_details(song_title, user_id, first_name, last_name)"
        query = query + "VALUES(%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [17]:
# Query to get the results from the table

event_details_query = "SELECT first_name, last_name FROM event_details WHERE song_title = %s"
try:
    rows = session.execute(event_details_query, ('All Hands Against His Own', ))
except Exception as e:
    print(e)
    
for row in rows:
    print("All Hands Against His Own is played by")
    print("User first_name: {}".format(row.first_name))
    print("User last_name: {}".format(row.last_name))
    print("******************************************************************************************************************************")

All Hands Against His Own is played by
User first_name: Jacqueline
User last_name: Lynch
******************************************************************************************************************************
All Hands Against His Own is played by
User first_name: Tegan
User last_name: Levine
******************************************************************************************************************************
All Hands Against His Own is played by
User first_name: Sara
User last_name: Johnson
******************************************************************************************************************************


### Drop the tables before closing out the sessions

In [18]:
session.execute("DROP TABLE IF EXISTS song_details")
session.execute("DROP TABLE IF EXISTS user_details")
session.execute("DROP TABLE IF EXISTS event_details")

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()